# Enrichment Analysis
![](./images/Module4/PathwayAnalysis.jpg)

Enrichment analysis (EA) is a technique used to derive biological insight from lists of significantly altered genes. The list of genes can be obtained from Differential Expression (DE) analysis or users' interest. The EA methods rely on the knowledge databases (e.g. KEGG, GO, ...) to identify biological pathways or terms that are enriched in a gene list more than would be expected by chance. The outcome of the EA would be the in-depth and contextualized findings to help understand the mechanisms of disease, genes and proteins associated with the etiology of a specific disease or drug target.

Over more than a decade, there are over 50 methods have been developed for EA. In this module, we will focus on pathway analysis using popular methods including Over Representation Analysis (ORA), Correlation Adjusted MEan RAnk (CAMERA), Kolmogorov-Smirnov (KS) test, Wilcoxon test, Fast Gene Set Enrichment Analysis (FGSEA), Gene Set Analysis (GSA), Significance Analysis of Function and Expression (SAFE), and Pathway Analysis with Down-weighting of Overlapping Genes (PADOG).

## Learning Objectives
1. Data preparation
2. Perform enrichment analysis using ORA, CAMERA, KS, Wilcoxon, FGSEA, GSA, SAFE, and PADOG
3. Visualize and interpret the outputs

## Necessary Files
In this submodule, users will have to provide the following inputs: 
- (1) Gene expression/count matrix in which, rows are genes and columns are samples and a differential analysis result, also in the form of a matrix. We recommend using gene ID as gene SYMBOL. If users use other different gene ID types, please refer to [our first submodule](./Module01-Data_Preparation.ipynb). 
for the instructions for gene mapping.
- (2) A table of sample information that has at least two columns: sample ID that matched the columns of the first input, and sample condition (e.g, control or disease)
- (3) A list of gene sets to used for the enrichment analysis. Please check out our [previous submodule](./Module01-Data_Preparation.ipynb) for more information of the format for the gene sets.

If users have been following our learning submodules, they only need to use the code available in the previous module to download the data from Google Cloud Bucket and load it for the analyses.

### Table of Contents

1. [Enrichment Analysis Using Over-Representation Analysis](#ora-main)
2. [Enrichment analysis using CAMERA](#camera-main)
3. [Enrichment analysis using KS and Wilcoxon test](#ks-main)
4. [Enrichment Analysis using FGSEA](#fgsea-main)
5. [Enrichment Analysis using GSA](#gsa-main)
6. [Enrichment analysis using SAFE](#safe-main)
7. [Enrichment analysis using PADOG](#padog-main)



In [84]:
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule4-1.html" width=100% height=250></iframe>')

### Data downloading
In this submodule, we will use the DE genes of `limma` analysis result of the `GSE5281` dataset from [**submodule 02** ](./Module02-DE_Analysis.ipynb) and the `.gmt` files containing GO terms, KEGG pathways, REACTOME pathways gene sets created in [**submodule 03** ](./Module03-Gene_Set_and_Pathway.ipynb) for performing enrichment analysis.
We can use the following command to download the data from Google Cloud Bucket.

In [85]:
# Download the GSE48350.rds file to the "data" folder in current directory
system("gsutil cp gs://ccpa_bucket/GSE5281.rds ./data/")

# Download the GO_terms.gmt, KEGG_pathways.gmt, REACTOME_pathways.gmt files to the "data" folder in current directory
system("gsutil cp gs://ccpa_bucket/GO_terms.gmt ./data/")
system("gsutil cp gs://ccpa_bucket/KEGG_pathways.gmt ./data/")
system("gsutil cp gs://ccpa_bucket/REACTOME_pathways.gmt ./data/")

Warning message in system("gsutil cp gs://ccpa_bucket/GSE48350.rds ./data/"):
“error in running command”
Warning message in system("gsutil cp gs://ccpa_bucket/GO_terms.gmt ./data/"):
“error in running command”
Warning message in system("gsutil cp gs://ccpa_bucket/KEGG_pathways.gmt ./data/"):
“error in running command”
Warning message in system("gsutil cp gs://ccpa_bucket/REACTOME_pathways.gmt ./data/"):
“error in running command”


We can now load the DE analysis result using the following commands.

In [49]:
# Load previously saved data
data <- readRDS("./data/GSE5281.rds")
# To see what are available in data
names(data)
DE.df <- data$limma_results
# Print some first row of the differential analysis result
head(DE.df)

[1] "expression_data" "samples"         "genes"           "limma_results"  
[5] "tTest_results"   "edgeR_results"   "deseq2_results"

,SYMBOL,p.value,pFDR,statistic,logFC,avgExpr
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TUBB,TUBB,1.432179e-22,3.177289e-18,-11.36377,-1.631276,9.123189
PSMB3,PSMB3,2.388997e-21,1.829653e-17,-10.92825,-1.408793,8.736749
TUBB4B,TUBB4B,2.474176e-21,1.829653e-17,-10.92281,-1.726316,10.297977
SLC35E1,SLC35E1,7.401763e-21,4.105203e-17,10.75253,1.627450,11.470567
ATP5F1C,ATP5F1C,9.950015e-21,4.414822e-17,-10.70648,-1.881757,10.012823
ATP5F1B,ATP5F1B,2.802296e-20,1.036149e-16,-10.54504,-1.616196,10.576243


<!-- headings -->
<a id="ora-main"></a>
## 1. Enrichment Analysis Using Over-Representation Analysis
Over-representation analysis (ORA) is a statistical method that determines whether genes from a pre-defined gene set of a specific GO term or KEGG pathway are presented more than would be expected (over-represented) in a subset of your data. In our learning module, this subset refers to the list of DE genes generated from the `limma` method. For each gene set, an enrichment *p-value* is calculated using the Binomial distribution, Hypergeometric distribution, the Fisher exact test, or the Chi-square test.  The hypergeometric distribution is a popular approach used to calculate enrichment p-value. The formula can be presented as follows:

$$
     P(X\geq x) = 1 - P(X \leq x-1) = 1 - \sum\limits_{i=0}^{x-1}\frac{\hphantom{}{M \choose i }{N - M \choose n-i}}{N \choose n}
$$

where *N* is the number of background genes (all genes presented in the expression matrix), *n* is the number of “interesting” genes (DE genes), *M* is the number of genes that are annotated to a particular gene set *S* (list of genes in a specific KEGG pathway or GO term), and *x* is the number of “interesting” genes that are annotated to *S* (genes presented in DE genes list and a specific KEGG pathway or GO term).

For example, suppose we have an expression matrix with 20,000 genes, of which 500 are differently expressed. Also, suppose that 100 of the 20,000 genes are annotated to a particular gene set S. Of these 100 genes, 20 are members of the DE genes list. The probability that 20 or more (up to 100) genes annotated to S are in the DE genes list by chance is given by

$$
P(X\geq 20) = 1 - P(X \leq 19) = 1-\sum \limits_{i=0}^{19}\frac{\hphantom{}{100 \choose i}{20000 - 100 \choose 500-i}}{20000 \choose 500} = 5.26 \times 10^{-13}
$$

The *p-value* indicates that it is very rare to observe 20 of the 100 genes from this set are in the DE genes list by chance.

In [7]:
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule4-2.html" width=100% height=250></iframe>')

### Data preparation
To conduct enrichment analysis using ORA, there are several input data that we need to prepare. First, we need to select a set of genes that
are significantly altered (*p-value < 0.05*) in the DE genes generated from the `limma` method.

In [8]:
# Selecting a list of significant DE genes
DEGenes <- DE.df[DE.df$pFDR <= 0.05, ]
# Select the remaining genes 
DEGenes <- DEGenes$SYMBOL

Next, we need to define a list of background genes. In this analysis, they are all the genes generated from the DE analysis.

In [9]:
# Defining background genes
backgroundSet <- DE.df$SYMBOL

Then, we need to obtain a list of geneset from knowledge databases such as GO and KEGG. In this learning module, the geneset will be retrieved from
the *.gmt* files that were processed from the [**submodule 03**](./Module03-Gene_Set_and_Pathway.ipynb). To load the geneset, we will use the `gmt2geneset` function below:

In [10]:
#' @description This function read a GMT file to a gene sets list in R
#'
#' @param path The file path to the GMT file.
#' @return A list containing the gene sets extracted from the GMT file.

gmt2geneset <- function(path) {
    # Read the lines from the file
    lines <- readLines(path, warn = FALSE)

    # Split each line into fields
    gmt_data <- strsplit(lines, "\t")

    # Process each line and extract gene sets
    gene_sets <- lapply(gmt_data, function(line) {
        gene_set_name <- line[1]
        description <- line[2]
        genes <- line[-c(1, 2)]  # Exclude the first two elements (name and description)

        list(
            pathwayID = gene_set_name,
            description = description,
            genes = genes
        )
    })
    # Convert list of gene sets to a named list
    gs <- lapply(gene_sets, function(g) g$genes)
    names(gs) <- lapply(gene_sets, function(g) g$pathwayID)

    return(gs)
}

Here, we also use a function called `runORA` to perform over-representation analysis based on the hyper-geometric testing formula presented above.
It requires a list of gene sets (E.g. GO terms, KEGG, REACTOME, ...), a dataframe containing results of a DE analysis method and a p-value threshold for a gene to be considered differentially expressed. The `ORA` method will perform hyper-geometric testing for each geneset obtained from GO or KEGG using the function `phyper` available for the `stats` R base package. The output of the `.runORA` function is a table that contains a column of terms or pathway names and a column of *p-value*.

In [11]:
#' @description This function performs gene set analysis based on ORA (Over Representation Analysis).
#'
#' @param DE_data The generated SummarizedExpriment object from DE analysis result.
#' @param genesets The gene to use for the analysis, e.g. KEGG gene sets from gmt2geneset function.
#' @param pThreshold The p.value cutoff threshold.
#' @return A dataframe of geneset analysis results

runORA <- function(DE_data, genesets, pThreshold = 0.05) {    
    # Check if DE_data is not null and has dimensions
    if (is.null(DE_data) |
        dim(DE_data)[1] == 0 |
        dim(DE_data)[2] == 0) {
        stop("No differential analysis data is in input data.")
    }

    # Extract genes with p-values below the threshold
    DE.genes <- rownames(DE_data[DE_data$p.value <= pThreshold, ])
        
    # All genes in the background
    background.genes <- rownames(DE_data)

    # Calculate the overlap between genesets and background genes
    GSOverlap <- sapply(genesets, function(gs) length(intersect(gs, background.genes)))
    # Calculate the overlap between genesets and differentially expressed genes
    DEOverlap <- sapply(genesets, function(gs) length(intersect(gs, DE.genes)))
    # Calculate the number of non-differentially expressed genes in the background
    NoneDEInBackground <- length(background.genes) - length(DE.genes)
    # Calculate the expected overlap by chance
    Expected <- GSOverlap * length(DE.genes) / length(background.genes)

    # Calculate p-values using hypergeometric distribution
    pvals <- 1 - phyper(DEOverlap - 1, length(DE.genes), NoneDEInBackground, GSOverlap)
    # Calculate Enrichment Score (ES)
    ES <- log2(DEOverlap / Expected)

    # Create a dataframe with results
    result_df <- data.frame(
        ID = names(genesets),
        p.value = pvals,
        score = ES,
        normalizedScore = ES, # For now, normalized score is the same as ES
        stringsAsFactors = FALSE
    )

    return(result_df)
}


### Enrichment analysis using ORA and GO terms
In this section, we will perform ORA using the list of gene sets obtained from the GO database and the function `ORA` defined above. The detailed code is presented below:

In [12]:
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/REACTOME_pathways.gmt")})})

In [13]:
# Loading GO terms
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/GO_terms.gmt")})})
# Perform ORA
ORA_res <- runORA(DE_data = DE.df, genesets = geneset, pThreshold = 0.05)

# Order the table based on the p-value
ORA_res <- ORA_res[order(ORA_res$p.value), ]
# View the most significant GO terms
head(ORA_res)

,ID,p.value,score,normalizedScore
,<chr>,<dbl>,<dbl>,<dbl>
GO:0006139,GO:0006139,0,0.2141292,0.2141292
GO:0006396,GO:0006396,0,0.4125171,0.4125171
GO:0006412,GO:0006412,0,0.5070574,0.5070574
GO:0006518,GO:0006518,0,0.4704241,0.4704241
GO:0006725,GO:0006725,0,0.2036541,0.2036541
GO:0006807,GO:0006807,0,0.2242210,0.2242210


From the result table, we might conclude that the pathways shown are key components of the disease in study.

### Enrichment analysis using ORA and KEGG pathways

We will perform the same steps as we did above with the GO terms.

In [14]:
# Loading KEGG pathways
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/KEGG_pathways.gmt")})})

# Perform Hyper-geometric testing
ORA_res <- runORA(DE_data = DE.df, genesets = geneset, pThreshold = 0.05)

# # Order the table based on the p-value
ORA_res <- ORA_res[order(ORA_res$p.value), ]
# View the most significant GO terms
head(ORA_res)


,ID,p.value,score,normalizedScore
,<chr>,<dbl>,<dbl>,<dbl>
hsa05012,hsa05012,0.000000e+00,0.8215426,0.8215426
hsa05014,hsa05014,0.000000e+00,0.6829997,0.6829997
hsa05016,hsa05016,0.000000e+00,0.7605060,0.7605060
hsa05022,hsa05022,4.440892e-16,0.5601445,0.5601445
hsa05020,hsa05020,7.549517e-15,0.6837973,0.6837973
hsa03050,hsa03050,2.984279e-13,1.2308632,1.2308632


### Enrichment analysis using ORA and REACTOME pathways

We will perform similar steps as we did before with the GO terms and KEEG pathways.

In [15]:
# Loading REACTOME pathways
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/REACTOME_pathways.gmt")})})
# Perform ORA
ORA_res <- runORA(DE_data = DE.df, genesets = geneset, pThreshold = 0.05)

# # Order the table based on the p-value
ORA_res <- ORA_res[order(ORA_res$p.value), ]
# View the most significant GO terms
head(ORA_res)

,ID,p.value,score,normalizedScore
,<chr>,<dbl>,<dbl>,<dbl>
R-HSA-72766,R-HSA-72766,0,0.7507580,0.7507580
R-HSA-163200,R-HSA-163200,0,0.9698630,0.9698630
R-HSA-199991,R-HSA-199991,0,0.5353433,0.5353433
R-HSA-392499,R-HSA-392499,0,0.3888270,0.3888270
R-HSA-611105,R-HSA-611105,0,1.0542352,1.0542352
R-HSA-1428517,R-HSA-1428517,0,0.8847324,0.8847324


<!-- headings -->
<a id="camera-main"></a>
## 2. Enrichment analysis using CAMERA

Correlation Adjusted MEan RAnk (CAMERA) is a powerful gene set analysis method designed for high-throughput genomic data. It operates in two stages, first calculating gene-level statistics including fold change, t-statistic, moderated t-statistics, and a normalized moderated t-statistic using a linear model. In the second stage, The inter-gene correlation within gene sets is assessed through the computation of the variance inflation factor (VIF), and this information is then integrated into either parametric or rank-based hypothesis tests.

Since we already had the gene-level statistics from DE analysis in submodule 2, We can utilize the `cameraPR` function integrated in `limma` package to perform the second stage of CAMERA. The `runCAMERA` function below is provided to perform analysis using CAMERA method.

In [14]:
# Import the limma package
library(limma)

In [15]:
#' @description This function performs gene set analysis using CAMERA method.
#'
#' @param DE_data A dataframe containing the DE analysis result.
#' @param genesets A list of gene sets to be analyzed.
#' @param stat_col A character string specifying the name of a statistic column in DE_data
#'                 to be used in the EA analysis.
#' @param use.ranks A logical whether to use parametric or rank-based hypothesis test
#' @return A dataframe containing the results of the tests for each gene set.

runCAMERA <- function(DE_data, genesets, stat_col = "statistic", use.ranks = FALSE) {
  
    # Extract the statistics column from DE_data
    stats <- DE_data[, stat_col]
    # Set gene names as row names for stats
    names(stats) <- rownames(DE_data)
    # Run CAMERA analysis using cameraPR function
    CAMERA_res <- cameraPR(statistic = stats, index = genesets, use.ranks = use.ranks)
    # Create a dataframe with results
    result_df <- data.frame(
        ID = rownames(CAMERA_res),
        p.value = CAMERA_res$PValue,
        pFDR = CAMERA_res$FDR,
        stringsAsFactors = FALSE
    )
    # Return the result dataframe
    result_df
}


### Enrichment analysis using CAMERA and GO terms
We will use the `runCAMERA` function to perform gene set analysis with GO terms as follows:

In [16]:
camera_start <- Sys.time()
# Loading GO terms
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/GO_terms.gmt")})})
# Perform EA using CAMERA method
CAMERA_res <- runCAMERA(DE_data = DE.df, genesets = geneset, stat_col = "statistic")
head(CAMERA_res)

,ID,p.value,pFDR
,<chr>,<dbl>,<dbl>
1,GO:0042773,7.796607e-19,2.693338e-15
2,GO:0042775,7.796607e-19,2.693338e-15
3,GO:0015986,1.351633e-17,3.112811e-14
4,GO:0019646,2.311338e-17,3.992258e-14
5,GO:0009145,3.894336e-16,5.381194e-13
6,GO:0006754,6.802849e-16,6.415202e-13


### Enrichment analysis using CAMERA and KEGG pathways

To analyze the KEGG pathways using CAMERA, we can use the same above codes and simply replace GO terms with KEGG pathways.

In [19]:
# Loading KEGG pathways
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/KEGG_pathways.gmt")})})
# Perform EA using CAMERA method
CAMERA_res <- runCAMERA(DE_data = DE.df, genesets = geneset, stat_col = "statistic")
head(CAMERA_res)

,ID,p.value,pFDR
,<chr>,<dbl>,<dbl>
1,hsa03050,2.620282e-21,9.013771e-19
2,hsa00190,2.075698e-18,3.570201e-16
3,hsa05012,4.282708e-15,4.910839e-13
4,hsa05016,1.491982e-13,1.283104e-11
5,hsa05020,1.004145e-12,6.908521e-11
6,hsa05014,2.878336e-12,1.650246e-10


### Enrichment analysis using CAMERA and REACTOME pathways

Similar to GO terms and KEGG pathways, we just need to load and pass the REACTOME pathways to the `runCAMERA` function.

In [20]:
# Loading REACTOME pathways
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/REACTOME_pathways.gmt")})})
# Perform EA using CAMERA method
CAMERA_res <- runCAMERA(DE_data = DE.df, genesets = geneset, stat_col = "statistic")
head(CAMERA_res)

camera_time <- Sys.time() - camera_start

,ID,p.value,pFDR
,<chr>,<dbl>,<dbl>
1,R-HSA-9604323,2.644210e-22,6.941051e-19
2,R-HSA-180585,1.032521e-21,1.318308e-18
3,R-HSA-9762114,1.506638e-21,1.318308e-18
4,R-HSA-211733,3.761629e-21,2.468569e-18
5,R-HSA-8854050,1.726367e-20,8.870846e-18
6,R-HSA-174113,2.314618e-20,8.870846e-18


<!-- headings -->
<a id="ks-main"></a>
## 3. Enrichment analysis using KS and Wilcoxon test

The Kolmogorov-Smirnov (KS) test is a non-parametric test that compares the cumulative distribution functions of two datasets. In the context of enrichment analysis, it can be employed to assess whether the distribution of DE genes is significantly different from what would be expected by chance for a given gene set. The test produces a KS statistic and a corresponding p-value, indicating the degree of over-representation.

The Wilcoxon test, also called the Mann-Whitney U test, helps us check if the ranks of certain genes (DE genes) are significantly different from the rest of the genes. It's a sturdy non-parametric method that doesn't rely on specific assumptions about data distribution. By looking at the ranks, it tells us if there are meaningful differences in how these genes are positioned compared to the overall set. In the context of enrichment analyses, the Wilcoxon test emerges as a valuable tool, elucidating critical variations in the hierarchical arrangement or ranking of genes within the dataset.

### Data preparation
To conduct enrichment analysis using KS or Wilcoxon test, we will use the limma DE result as input. First, we can use the DE.df variable above and select a set of genes that
are significantly altered (*p-value < 0.05*). 

In [21]:
# Select genes that are significantly altered
kw_de <- DE.df[DE.df$pFDR <= 0.05, ]
head(kw_de)

,SYMBOL,p.value,pFDR,statistic,logFC,avgExpr
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TUBB,TUBB,1.432179e-22,3.177289e-18,-11.36377,-1.631276,9.123189
PSMB3,PSMB3,2.388997e-21,1.829653e-17,-10.92825,-1.408793,8.736749
TUBB4B,TUBB4B,2.474176e-21,1.829653e-17,-10.92281,-1.726316,10.297977
SLC35E1,SLC35E1,7.401763e-21,4.105203e-17,10.75253,1.627450,11.470567
ATP5F1C,ATP5F1C,9.950015e-21,4.414822e-17,-10.70648,-1.881757,10.012823
ATP5F1B,ATP5F1B,2.802296e-20,1.036149e-16,-10.54504,-1.616196,10.576243


Next, we will remove rows containing NA, dupicated symbols.

In [22]:
# Remove NA value
kw_de <- kw_de[!is.na(kw_de$SYMBOL), ]
# Remove duplicated genes symbols
kw_de <- kw_de[!duplicated(kw_de$SYMBOL, fromLast = FALSE), ]
rownames(kw_de) <- kw_de$SYMBOL
# remove the first column from the DE dataframe  
kw_de <- kw_de[, !(names(kw_de) %in% c("SYMBOL"))]

head(kw_de)

,p.value,pFDR,statistic,logFC,avgExpr
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TUBB,1.432179e-22,3.177289e-18,-11.36377,-1.631276,9.123189
PSMB3,2.388997e-21,1.829653e-17,-10.92825,-1.408793,8.736749
TUBB4B,2.474176e-21,1.829653e-17,-10.92281,-1.726316,10.297977
SLC35E1,7.401763e-21,4.105203e-17,10.75253,1.627450,11.470567
ATP5F1C,9.950015e-21,4.414822e-17,-10.70648,-1.881757,10.012823
ATP5F1B,2.802296e-20,1.036149e-16,-10.54504,-1.616196,10.576243


To perform KS or Wilcoxon test, we will use the `runKsWilcox` function bellow.

In [23]:
#' Run Kolmogorov-Smirnov Wilcoxon rank-sum test on differential expression data
#'
#' @param DE_data A dataframe containing the DE analysis result.
#' @param statCol A character string specifying the name of a statistic column in DE_data to be used in the EA analysis.
#' @param genesets A list of gene sets to be analyzed.
#' @param sTest The type of test to be used ("ks" for Kolmogorov-Smirnov, "wilcox" for Wilcoxon rank-sum).
#' @return A dataframe containing the results of the tests for each gene set.

runKsWilcox <- function(DE_data, genesets, statCol = "statistic", sTest = "ks") {
    # remove rows that contain NA value in the column specified by the statCol argument
    DE_data <- DE_data[!is.na(DE_data[[statCol]]), ]
    # get the statistical values from DE_data to perform the test
    ranks <- DE_data[[statCol]]
    # set names of the DE genes
    names(ranks) <- rownames(DE_data)

    # choose the test to perform based on the sTest argument
    test <- if (sTest == "ks") ks.test else wilcox.test
    
    background.genes <- rownames(DE_data)

    DEhit <- sapply(genesets, function(gs) ranks[background.genes[background.genes %in% gs]])
    DEmiss <- sapply(genesets, function(gs) ranks[background.genes[!background.genes %in% gs]])

    # Initialize an empty list to store results
    PA_res_list <- lapply(1:length(DEhit), function(i) {
        cur.DEhit <- DEhit[[i]]
        cur.DEmiss <- DEmiss[[i]]
    
        # Check if either list is empty
        if (length(cur.DEhit) == 0 | length(cur.DEmiss) == 0) return(1)
    
        # Perform the test and extract the p-value
        test_result <- test(cur.DEhit, cur.DEmiss)
        test_result$p.value
    })
    
    # Combine the results into a data frame
    PA_res <- data.frame(
        ID = names(genesets),
        p.value = unlist(PA_res_list),
        stringsAsFactors = FALSE
    )
    
    # Remove rows with missing values
    PA_res <- PA_res[complete.cases(PA_res), ]

    return(PA_res)
}

### Enrichment analysis using KS and GO terms

We will conduct KS enrichment analysis with GO terms gene sets list using the provided code.

In [24]:
# load the GO gene sets from saved gmt file
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/GO_terms.gmt")})})
# perform enrichment analysis using KS test
suppressWarnings(k_results <- runKsWilcox(DE_data = kw_de, geneset = geneset, statCol = "statistic", sTest = "ks"))

# order the results 
k_results <- k_results[order(k_results$p.value), ]
# show some first rows of the result table
head(k_results)

,ID,p.value
,<chr>,<dbl>
628,GO:0006091,0
1183,GO:0008150,0
1248,GO:0009060,0
2989,GO:0035239,0
2999,GO:0035295,0
3708,GO:0045333,0


### Enrichment analysis using KS and KEGG pathways
To do the analysis with KEEG pathways, we will replace the GO gene sets list with the KEGG pathways.

In [110]:
# Loading the KEGG pathways from saved gmt file
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/KEGG_pathways.gmt")})})
# perform enrichment analysis using KS test
k_results <- runKsWilcox(DE_data = kw_de, geneset = geneset, statCol = "statistic", sTest = "ks")

# order the results 
k_results <- k_results[order(k_results$p.value), ]
# show some first rows of the result table
head(k_results)

,ID,p.value
,<chr>,<dbl>
172,hsa04610,0.003411117
165,hsa04520,0.031746032
289,hsa05168,0.043373174
207,hsa04918,0.055555556
171,hsa04640,0.071333713
181,hsa04612,0.071333713


### Enrichment analysis using KS and REACTOME pathways
To do the analysis with REACTOME pathways, we will use the same steps as for GO terms and KEEG pathways.

In [111]:
# Loading the KEGG pathways from saved gmt file
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/REACTOME_pathways.gmt")})})
# perform enrichment analysis using KS test
k_results <- runKsWilcox(DE_data = kw_de, geneset = geneset, statCol = "statistic", sTest = "ks")

# order the results 
k_results <- k_results[order(k_results$p.value), ]
# show some first rows of the result table
head(k_results)

,ID,p.value
,<chr>,<dbl>
612,R-HSA-210744,0.007936508
615,R-HSA-210747,0.007936508
1870,R-HSA-5690714,0.008089547
797,R-HSA-425366,0.015873016
801,R-HSA-425407,0.015873016
855,R-HSA-444411,0.015873016


### Enrichment analysis using WILCOXON and GO terms

For enrichment analysis with Wilcoxon test, we will pass the value "wilcox" to the *sTest* argument in the `runKsWilcox` function. We will do the same steps as KS test.

In [112]:
# load the GO gene sets from saved gmt file
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/GO_terms.gmt")})})
# perform enrichment analysis with Wilcoxon test 
w_results <- runKsWilcox(DE_data = kw_de, geneset = geneset, statCol = "statistic", sTest = "wilcox")

# order the result  
w_results <- w_results[order(w_results$p.value), ]
# show some first rows of the result
head(w_results)

,ID,p.value
,<chr>,<dbl>
3864,GO:0045935,0.007026383
5701,GO:0090303,0.007074130
6399,GO:1903036,0.007074130
5446,GO:0071840,0.007700341
3846,GO:0045893,0.008484490
4519,GO:0051254,0.008484490


### Enrichment analysis using WILCOXON and KEGG pathways
To do the analysis with KEEG pathways, we will replace the GO gene sets with the KEGG pathways.

In [113]:
# Loading the KEGG pathways from saved gmt file
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/KEGG_pathways.gmt")})})
# perform enrichment analysis with Wilcoxon test 
w_results <- runKsWilcox(DE_data = kw_de, geneset = geneset, statCol = "statistic", sTest = "wilcox")

# order the result  
w_results <- w_results[order(w_results$p.value), ]
# show some first rows of the result
head(w_results)

,ID,p.value
,<chr>,<dbl>
172,hsa04610,0.01234620
289,hsa05168,0.04057075
181,hsa04612,0.05563734
303,hsa05152,0.05563734
171,hsa04640,0.09105652
165,hsa04520,0.09352453


### Enrichment analysis using WILCOXON and REACTOME pathways
We can use the same code for REACTOME pathways.

In [114]:
# Loading the KEGG pathways from saved gmt file
suppressWarnings({suppressMessages({geneset <- gmt2geneset("./data/REACTOME_pathways.gmt")})})

# perform enrichment analysis using KS test
w_results <- runKsWilcox(DE_data = kw_de, geneset = geneset, statCol = "statistic", sTest = "wilcox")

# order the results 
w_results <- w_results[order(w_results$p.value), ]
# show some first rows of the result table
head(w_results)

,ID,p.value
,<chr>,<dbl>
1870,R-HSA-5690714,0.01795096
999,R-HSA-983695,0.02620906
2440,R-HSA-9679191,0.02620906
1138,R-HSA-1640170,0.03103296
1139,R-HSA-1643685,0.03490709
874,R-HSA-446728,0.04057075


<!-- headings -->
<a id="fgsea-main"></a>
## 4. Enrichment Analysis using FGSEA

This section describes `FGSEA`, one of the methods for evaluating pathway enrichment in transcriptional data and it stands for Fast preranked Gene Set Enrichment Analysis (GSEA). `FGSEA` quickly calculates arbitrarily low GSEA P-values for a collection of gene sets and is based on an algorithm that allows  it to make more permutations and get accurate *p-values*.
It extends the GSEA algorithm which calculates the enrichment score (the degree to which a set S is over-represented at the top or bottom of the ranked list L), estimates the p-value of the Enrichment Score using a permutation test and adjusts the estimated significance level to account for multiple hypotheses in addition to calculating the q-values for FDR control. Typically, GSEA requires inputs of a list of gene sets (GO term or pathway with a set of genes), a vector DE genes with statistic. In addition, users can limit the number of terms or pathways of interest by adjusting the `minSize` and `maxSize` parameters. The code to install and load the FGSEA R package is shown below:

In [17]:
# To perform enrichment analysis using FGSEA, we first install the fgsea package
suppressMessages({
    if (!require("BiocManager", quietly = TRUE)) {
        install.packages("BiocManager")
    }
    suppressWarnings(BiocManager::install("fgsea", update = F))
    
})
# Loading the package
suppressPackageStartupMessages({
    library("fgsea")
})

# Get the number of maximum cores for parallel processing
library(parallel)
ncores <- detectCores()

# Set number of permutation for FGSEA
nperms <- 2000

For conducting pathway analysis using FGSEA, we can use the runFGSEA function written below. This function needs 

In [18]:
#' @description This function performs gene set analysis using fgsea (fast gene set enrichment analysis).
#'
#' @param DE_data A dataframe containing the DE analysis result.
#' @param genesets A list of gene sets to be analyzed, ex. KEGG genesets.
#' @param statCol A character string specifying the name of a statistic column in DE_data to be used in the EA analysis.
#' @param nperms An integer specifying the number of permutations for fgsea. 
#' @param ncores An integer specifying the number cpu cores for parallel processing.
#' @return A list containing a dataframe of gene set analysis results and a statistic vector

runFgsea <- function(DE_data, genesets, statCol = "statistic", nperms = 200, ncores = 0) {

    if (is.null(DE_data) |
        dim(DE_data)[1] == 0 |
        dim(DE_data)[2] == 0) {
        stop("No differential analysis data is in input data.")
    }

    DE_data <- DE_data[!is.na(DE_data[[statCol]]),]

    statistic <- unlist(DE_data[[statCol]]) 
    statistic <- as.vector(statistic)
    names(statistic) <- rownames(DE_data)

    suppressWarnings(fgsea_res <- fgsea::fgsea(pathways = genesets, stats = statistic, nperm = nperms, nproc = ncores))

    fgsea_res <- fgsea_res[!is.na(fgsea_res[["pval"]]), ]
    fgsea_res$ID <- fgsea_res$pathway
    fgsea_res$p.value <- fgsea_res$pval
    fgsea_res$score <- fgsea_res$ES
    fgsea_res$normalizedScore <- fgsea_res$NES
    
    list(fgsea_res = fgsea_res, stats = statistic)
}

In [19]:
#Run the following command to take the quiz
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule4-3.html" width=100% height=250></iframe>')

Next, we need to prepare a vector of DE genes with its statistic.

### Enrichment analysis using FGSEA and GO terms

The package `fgsea` has its own function named `gmtPathways` to load the genesets from the `gmt` file. It is recommended to use this function for compatibility. We can load the GO terms geneset and view the first five GO terms with associated genes using the following command:

In [20]:
# Load the pathways into a named list
GO_term_hallmark <- gmtPathways("./data/GO_terms.gmt")
# Show the first few GO terms, and within those, show only the first few genes.
tmp = lapply(GO_term_hallmark, head)
tmp[1:5]

$`GO:0000002`
[1] "AKT3"   "DNA2"   "DNAJA3" "ENDOG"  "FLCN"   "LIG3"  

$`GO:0000003`
[1] "A1CF"  "A2M"   "AAAS"  "ABAT"  "ABCC8" "ABHD2"

$`GO:0000012`
[1] "APLF"  "APTX"  "ERCC6" "ERCC8" "LIG4"  "PARP1"

$`GO:0000018`
[1] "ABL1"   "ACTB"   "ACTL6A" "ACTR2"  "ALYREF" "ANKLE1"

$`GO:0000022`
[1] "AURKB"  "AURKC"  "BIRC5"  "CDCA8"  "INCENP" "KIF23"

Running the FGSEA can be done by calling the `fgsea` with two required inputs that we created from the two previous steps.

In [119]:
# Running fgsea analysis
geneset <- GO_term_hallmark
fgsea_output <- runFgsea(DE_data = DE.df, genesets = geneset, nperms = nperms, ncores = ncores)
# Get result and stats from output of runFgsea function
fgsea_result <- fgsea_output$fgsea_res
stats <- fgsea_output$stats

head(fgsea_result[order(fgsea_result$p.value), c("ID", "p.value", "score", "normalizedScore")])

From the result table, we can select top five up regulated GO terms and top five down regulated GO terms. Then we can plot them using the built-in function `plotGseaTable`

In [120]:
# select the top five up-regulated pathways 
topPathwaysUp <- fgsea_result[ES > 0][head(order(pval), n = 5), pathway]
# select the top five down-regulated pathways 
topPathwaysDown <- fgsea_result[ES < 0][head(order(pval), n = 5), pathway]
# concatenate them for visualization
topPathways <- c(topPathwaysUp, rev(topPathwaysDown))
#Viewing the 5 most significantly up-regulated and down-regulated pathways each with the FGSEA internal plot function
plotGseaTable(pathways = GO_term_hallmark[topPathways], stats = stats, 
              fgseaRes = fgsea_result, gseaParam = 0.5)

To use the `plotGseaTable` function, we need to pass the following arguments:

- `pathways`: A list of pathways to plot table.
- `stats`: A dataframe specifying gene-level stats.
- `fgseaRes`: A dataframe of fgsea results.
- `gseaParam`: A numeric value to adjust displayed statistic values, values closer to 0 flatten plots. Default = 1, value of 0.5 is a good choice too.

### Enrichment analysis using FGSEA and KEGG pathways
We can perform enrichment analysis using FGSEA with KEGG pathway using the same procedure mentioned above. The only thing we need to change is the
list of gene sets that are available in the KEGG database.

In [121]:
# Load the pathways into a named list
KEGG_hallmark <- gmtPathways("./data/KEGG_pathways.gmt")
# Show the first few GO terms, and within those, show only the first few genes.
tmp = lapply(KEGG_hallmark, head)
tmp[1:5]

In [122]:
# Running fgsea analysis
geneset <- KEGG_hallmark
fgsea_output <- runFgsea(DE_data = DE.df, genesets = geneset, nperms = nperms, ncores = ncores)
# Get result and stats from output of runFgsea function
fgsea_result <- fgsea_output$fgsea_res
stats <- fgsea_output$stats

head(fgsea_result[order(fgsea_result$p.value), c("ID", "p.value", "score", "normalizedScore")])

In [123]:
# select the top five up-regulated pathways 
topPathwaysUp <- fgsea_result[ES > 0][head(order(pval), n = 5), pathway]
# select the top five down-regulated pathways 
topPathwaysDown <- fgsea_result[ES < 0][head(order(pval), n = 5), pathway]
# concatenate them for visualization
topPathways <- c(topPathwaysUp, rev(topPathwaysDown))
#Viewing the 5 most significantly up-regulated and down-regulated pathways each with the FGSEA internal plot function
plotGseaTable(pathways = KEGG_hallmark[topPathways], stats = stats, 
              fgseaRes = fgsea_result, gseaParam = 0.5)

### Enrichment analysis using FGSEA and REACTOME pathways
We can perform enrichment analysis using FGSEA with REACTOME pathways using the same procedure above. 

In [124]:
# Load the pathways into a named list
REACTOME_hallmark <- gmtPathways("./data/REACTOME_pathways.gmt")
# Show the first few GO terms, and within those, show only the first few genes.
tmp = lapply(REACTOME_hallmark, head)
tmp[1:5]

In [125]:
# Running fgsea analysis
geneset <- REACTOME_hallmark
fgsea_output <- runFgsea(DE_data = DE.df, genesets = geneset, nperms = nperms, ncores = ncores)
# Get result and stats from output of runFgsea function
fgsea_result <- fgsea_output$fgsea_res
stats <- fgsea_output$stats

head(fgsea_result[order(fgsea_result$p.value), c("ID", "p.value", "score", "normalizedScore")])

In [126]:
# select the top five up-regulated pathways 
topPathwaysUp <- fgsea_result[ES > 0][head(order(pval), n = 5), pathway]
# select the top five down-regulated pathways 
topPathwaysDown <- fgsea_result[ES < 0][head(order(pval), n = 5), pathway]
# concatenate them for visualization
topPathways <- c(topPathwaysUp, rev(topPathwaysDown))
#Viewing the 5 most significantly up-regulated and down-regulated pathways each with the FGSEA internal plot function
plotGseaTable(pathways = REACTOME_hallmark[topPathways], stats = stats, 
              fgseaRes = fgsea_result, gseaParam = 0.5)

<!-- headings -->
<a id="gsa-main"></a>
## 5. Enrichment Analysis using GSA

Gene Set Analysis (GSA), an Enrichment Analysis, is a method that is commonly used to summarize high-dimensional gene expression data sets into sets according to their biological relevance. GSA takes the ranked gene lists from the initial stage of a gene expression analysis and aggregates the genes into sets based on shared biological or functional properties as specified by a reference knowledge base. Such databases often contain phenotype associations, molecular interactions and regulation and are referenced in the analysis of the resultant gene sets to find the relevance of the gene properties to the phenotype of interest.

In [21]:
#Run the following command to take the quiz
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule4-4.html" width=100% height=250></iframe>')

### Data preparation
The GSA method is freely available as standalone package in CRAN repository. We can use the following code to install the package.

In [40]:
# Install GSA from CRAN
suppressMessages({
    suppressWarnings({
        if (!require("GSA")) install.packages("GSA")
    })
})

suppressMessages({
    suppressWarnings({
        library(GSA)
    })
})


GSA employs a permutation model to estimate p-values for each gene set. The precision of these estimates increases with a larger number of permutations, but this also necessitates longer running times. The GSA package available on CRAN lacks parallel processing support for this task. To address this, we've modified the original GSA function within the GSA package to enable multi-core processing for p-value estimation using the permutation model. The updated code is stored in the GSAMC.R script and can be imported using the following commands:

In [41]:
# GSA function for parallel processing
source("./GSAMC.R")
nperms <- 2000


In [42]:
#' @description This function performs enrichment analysis using the GSA method.
#'
#' @param exprsData A numeric matrix or a matrix-like object containing gene expression data. Rows represent genes, columns represent samples.
#' @param genesets A list of gene sets to be analzyed.
#' @param groups A factor indicating sample groups.
#' @param parallel A boolean to choose between non-parallel and parallel processing.
#' @param ncores An integer specifying the number of cores to use for parallel processing (default is 100).
#' @param nperms An integer specifying the number of permutations for assessing significance (default is 100).
#'
#' @return A list containing the GSA results data frame and the GSA object.

runGSA <- function(exprsData, genesets, groups, parallel = FALSE, ncores = 2, nperms = 100) {
    # Determine the number of classes in the group variable
    num_classes <- length(unique(groups))
    resp_type <- NULL

    # Check the number of classes and set response type accordingly
    if (num_classes == 1) {
        stop("The group classes in the design matrix must be at least two.")
    } else if (num_classes == 2) {
        resp_type <- "Two class unpaired"
    } else {
        resp_type <- "Multiclass"
    }
    # Run GSA analysis
    if (parallel) {
        gsa_res <- GSAMC(
            x = exprsData,
            y = groups,
            genesets = genesets,
            resp.type = resp_type,
            genenames = rownames(exprsData),
            nperms = nperms,
            ncores = ncores
        )
    } else {
        gsa_res <- GSA::GSA(
            x = exprsData,
            y = groups,
            genesets = genesets,
            resp.type = resp_type,
            genenames = rownames(exprsData),
            nperms = nperms
        )
    }
    # Extract p-values and create result data frame
    pvalues <- apply(cbind(gsa_res$pvalues.lo, gsa_res$pvalues.hi), 1, min)
    res <- data.frame(ID = names(genesets), p.value = pvalues * 2, stringsAsFactors = FALSE)

    # Create GSA result data frame
    GSA_res <- data.frame(
        ID = res$ID,
        p.value = res$p.value,
        score = gsa_res$GSA.scores,
        normalizedScore = gsa_res$GSA.scores,
        stringsAsFactors = FALSE
    )
    # Return a list containing GSA results
    list(GSA_res = GSA_res[!is.na(GSA_res$p.value), ], GSA.obj = gsa_res)
}


The GSA method requires an expression matrix, a numeric vector containing the class of each sample and a vector of the genes inputs. We can easily get those inputs by loading the data that we processed in the [**submodule 01** ](./Module01-GEO_Data_Processing.ipynb). From the samples information, we can use the same code in [**submodule 02** ](./Module02-DE_Analysis.ipynb) to create the groups information factor.

In [43]:
names(data)

[1] "expression_data" "samples"         "genes"           "limma_results"  
[5] "tTest_results"   "edgeR_results"   "deseq2_results"

In [44]:
# Loading expression data and samples information
expression_data <- data$expression_data
norm_expression_data <- log2(expression_data + 1)
samples <- data$samples


Get the groups information for GSA function.

In [45]:
groups <- samples$condition
groups <- as.factor(groups)
groups

[1] c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c
 [38] c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c
 [75] d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d
[112] d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d
[149] d d d d d d d d d d d d d
Levels: c d

### Enrichment Analysis using GSA and GO Terms
Using data obtained from the previous step, we can run the GSA method by calling the function `GSA`. We can reuse `GO_term_hallmark` and `KEGG_hallmark` loaded in FGSEA to perform analysis. The code details are shown below:

In [46]:
genesets <- GO_term_hallmark
GSA_output <- runGSA(as.matrix(norm_expression_data), genesets = genesets, as.numeric(groups), parallel = TRUE, ncores = ncores, nperms = nperms)
# Get GSA_result dataframe and GSA object from output of the runGSA function
GSA_result <- GSA_output$GSA_res
GSA.obj <- GSA_output$GSA.obj

In [135]:
head(GSA_result[order(GSA_result$p.value), ])

<div class="alert alert-block alert-danger">
    <i class="fa fa-exclamation-circle" aria-hidden="true"></i>
    <b>Alert: </b>  Please note that the outputs generated from GSA can be varied depending on R environments and software versions. Especially, when the users run the scripts using User-managed Notebooks instances which have a preinstalled suite of packages.
</div>

From the analysis result, we can now get the list positive and negative genes of the GSA analysis from the GSA object using the `GSA.listsets` function. 

In [136]:
# List the results from a GSA analysis
res <- GSA.listsets(GSA.obj = GSA.obj, geneset.names = names(genesets), FDRcut = .5)

To use the `GSA.listsets` function, we need to pass the following arguments:

- `GSA.obj`: A GSA object returned by GSA function.
- `geneset.names`: An optional vector of names for the gene sets.
- `FDRcut`: A numeric value specifying the false discovery rate cutpoint for listed sets. A value of 1 will cause all sets to be listed.

The `GSA.listsets` function returns a list of the results from the GSA call.

A table of the negative gene sets. “Negative” means that lower expression of most genes in the gene set correlates with higher values of the phenotype y. E.g., for two classes coded 1,2, lower expression correlates with class 2.

In [137]:
neg.table <- res$negative
head(neg.table)

A table of the positive gene sets. "Positive" means that higher expression of most genes in the gene set correlates with higher values of the phenotype y.

In [138]:
pos.table <- res$positive
head(pos.table)

In [139]:
# Individual gene scores from a gene set analysis
# look at 10th gene set
GSA.genescores(geneset.number = 10, genesets = genesets, 
               GSA.obj = GSA.obj, genenames = genenames)

To use the `GSA.genescores` function, we need to pass the following arguments:

- `geneset.number`: A number specifying which gene set is to be examined.
- `genesets`: A list containing the gene set collection.
- `GSA.obj`: A GSA object returned by the GSA function.
- `genenames`: A vector containing gene names for genes in the expression dataset.

The `GSA.genescores` returns a matrix of gene names and gene scores (eg t-statistics) for each gene in the gene set.


In [140]:
# Plot the result, this function makes a plot of the significant gene sets, based on a call to the GSA (Gene set analysis) function.
suppressWarnings(GSA.plot(GSA.obj, fac = 1, FDRcut = 0.5))

To use the `GSA.plot` function, we need to pass the following arguments:

- `GSA.obj`: A GSA object returned by the GSA function.
- `fac`: A numeric value for jittering points in plot ("factor" in called to jitter()).
- `FDRcut`: A numeric value specifying the false discovery rate cutpoint for sets to be plotted. A value of 1 (the default) will cause all sets to be plotted.



### Enrichment analysis using GSA and KEGG pathways
We can use the same procedure to per enrichment analysis with the KEGG pathway. All the codes are similar but `genesets` are assigned from `KEGG_hallmark`. The code is shown below.

In [141]:
genesets <- KEGG_hallmark
GSA_output <- runGSA(as.matrix(new_expression_data), genesets = genesets, as.numeric(groups), parallel = TRUE, nperms = nperms)
# Get GSA_result dataframe and GSA object from output of the runGSA function
GSA_result <- GSA_output$GSA_res
GSA.obj <- GSA_output$GSA.obj

In [142]:
head(GSA_result[order(GSA_result$p.value), ])

,ID,p.value,score,normalizedScore
,<chr>,<dbl>,<dbl>,<dbl>
2,R-HSA-68616,0,0.1565245,0.1565245
5,R-HSA-68875,0,0.1172641,0.1172641
9,R-HSA-68884,0,1.0225575,1.0225575
12,R-HSA-68949,0,-0.5636263,-0.5636263
16,R-HSA-69017,0,-0.6209957,-0.6209957
17,R-HSA-69052,0,-0.5047678,-0.5047678


In [143]:
# List the results from a GSA analysis
res <- GSA.listsets(GSA.obj = GSA.obj, geneset.names = names(genesets), FDRcut = .5)

A table of the negative gene sets. "Negative" means that lower expression of most genes in the gene set correlates with higher values of the phenotype y. E.g., for two classes coded 1,2, lower expression correlates with class 2.

In [144]:
neg.table <- res$negative
head(neg.table)

A table of the positive gene sets. "Positive" means that higher expression of most genes in the gene set correlates with higher values of the phenotype y. See "negative" above for more info.

In [145]:
pos.table <- res$positive
head(pos.table)

In [146]:
# Individual gene scores from a gene set analysis
# look at 10th gene set
GSA.genescores(geneset.number = 10, genesets = genesets, 
               GSA.obj = GSA.obj, genenames = genenames)

In [147]:
# Plot the result, this function makes a plot of the significant gene sets, based on a call to the GSA (Gene set analysis) function.
suppressWarnings(GSA.plot(GSA.obj, fac = 1, FDRcut = 0.5))


### Enrichment analysis using GSA and REACTOME pathways
We can use the same procedure to per enrichment analysis with the REACTOME pathway. All the codes are similar but `genesets` are assigned from `REACTOME_hallmark`. The code is shown below.

In [148]:
genesets <- REACTOME_hallmark
GSA_output <- runGSA(as.matrix(new_expression_data), genesets = genesets, as.numeric(groups), parallel = TRUE, nperms = nperms)
# Get GSA_result dataframe and GSA object from output of the runGSA function
GSA_result <- GSA_output$GSA_res
GSA.obj <- GSA_output$GSA.obj

In [149]:
head(GSA_result[order(GSA_result$p.value), ])


In [150]:
# List the results from a GSA analysis
res <- GSA.listsets(GSA.obj = GSA.obj, geneset.names = names(genesets), FDRcut = .5)

In [151]:
neg.table <- res$negative
head(neg.table)

In [152]:
pos.table <- res$positive
head(pos.table)

In [153]:
# Individual gene scores from a gene set analysis
# look at 10th gene set
GSA.genescores(geneset.number = 10, genesets = genesets, 
               GSA.obj = GSA.obj, genenames = genenames)

In [154]:
# Plot the result, this function makes a plot of the significant gene sets, based on a call to the GSA (Gene set analysis) function.
suppressWarnings(GSA.plot(GSA.obj, fac = 1, FDRcut = 0.5))


<!-- headings -->
<a id="safe-main"></a>
## 6. Enrichment analysis using SAFE


Similar to the CAMERA method, the Significance Analysis of Function and Expression (SAFE) method also has two stages. First, it calculates local statistics to measure the significance of each gene's association in the dataset, using tests like the Student’s t-test, Welch’s t-test, t-statistic from a linear model, etc. In the second stage, it computes global statistics to assess the significance of a competitive hypothesis test for each gene set or pathway. These global statistics, including Wilcoxon rank sum, Fisher’s Exact Test, Pearson’s Chi-squared type statistic, and a t-statistic for average difference, help compare various gene sets. To control the false positives rate, SAFE uses permutation of class labels.

We can use the function `safe` from the bioconductor `safe` package to perform EA using this method. We first need to install and import the required packages.

In [155]:
# Install and import safe 
suppressWarnings({
    suppressMessages({
        if (!require("BiocManager", quietly = TRUE))
            install.packages("BiocManager")
        
        BiocManager::install("safe")
        
        library(safe)
        library(hgu133plus2.db)
    })
})
library(doParallel) 

registerDoParallel(cores = ncores)
nperms <- 2000

The `runSAFE` function below requires a gene expression matrix, a list of gene sets, and the group information of the expression matrix. It utilizes the `getCmatrix` and `safe` functions to perform enrichment analysis using the SAFE method. The `getCmatrix` function creates a matrix from a genes list of a dataset and a list of gene sets. This matrix will be used as an input of the `safe` function. 

In [156]:
#' @description This function performs enrichment analysis using the SAFE method.
#'
#' @param exprsData A numeric matrix or a matrix-like object containing gene expression data. Rows represent genes, columns represent samples.
#' @param genesets A list of gene sets to be analyzed.
#' @param groups A factor indicating sample groups.
#' @param nperms An integer specifying the number of permutations for assessing significance (default is 1000).
#' @param seed An integer specifying a seed for reproducibility of permutation results (default is 1).
#'
#' @return A data frame containing pathway IDs and corresponding p-values.
runSAFE <- function(exprsData, geneSet, groups, nperms = 1000, parallel = TRUE, seed = 42) { 
    set.seed(seed)
    # Create the C matrix from the list of gene sets and list of gene IDs in the expression data
    C.mat2<- getCmatrix(keyword.list = geneSet, present.genes = rownames(exprsData))
    # Perform enrichment analysis using the safe function
    results<- safe(as.matrix(exprsData), groups, C.mat = C.mat2, 
                   platform = "hgu133plus2.db", Pi.mat = nperms, 
                   parallel = parallel, print.it =FALSE)
    # Extract the analysis result
    res <- safe.toptable(results, number = length(geneset))
    return(res)
}

### Enrichment analysis using SAFE and GO terms 
We can perform SAFE enrichment analysis with GO terms using the following commands.

In [157]:
# Get the GO terms, which is previously loaded for the FGSEA method
geneset <- GO_term_hallmark
# Perform the analysis using SAFE
res <- runSAFE(exprsData = new_expression_data, geneSet = geneset, groups = groups, nperms = nperms)
head(res)

6909 categories formed


,GenesetID,Size,Statistic,P.value,Adj.p.value,Description
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,GO:0006793,2569,31148494,0.1,0.8793,phosphorus metabolic process
2,GO:0006796,2542,30819767,0.1,0.8793,phosphate-containing compound metabolic process
3,GO:0044281,1869,22335985,0.1,0.8793,small molecule metabolic process
4,GO:0044093,1291,15547409,0.1,0.8793,positive regulation of molecular function
5,GO:0051174,1198,14368506,0.1,0.8793,regulation of phosphorus metabolic process
6,GO:0019220,1197,14363314,0.1,0.8793,regulation of phosphate metabolic process


### Enrichment analysis using SAFE and KEGG pathways
We can use the same command as for GO terms. We just need to pass the KEGG pathways instead of GO terms.

In [158]:
# Get the KEGG pathways
geneset <- KEGG_hallmark
# Perform the analysis using SAFE
res <- runSAFE(exprsData = new_expression_data, geneSet = geneset, groups = groups, nperms = nperms)
head(res)

345 categories formed


,GenesetID,Size,Statistic,P.value,Adj.p.value,Description
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,hsa05022,463,6393856,0.1,0.8086,NA
2,hsa05010,372,5119770,0.1,0.8086,NA
3,hsa05014,353,4929517,0.1,0.8086,NA
4,hsa05016,295,4436718,0.1,0.8086,NA
5,hsa05020,263,3896002,0.1,0.8086,NA
6,hsa05012,258,3876468,0.1,0.8086,NA


### Enrichment analysis using SAFE and REACTOME pathways 
We can do the same as GO terms and KEGG pathways. 

In [159]:
# Get the REACTOME pathways
geneset <- REACTOME_hallmark
# Perform the analysis using SAFE
res <- runSAFE(exprsData = new_expression_data, geneSet = geneset, groups = groups, nperms = nperms)
head(res)



2358 categories formed


,GenesetID,Size,Statistic,P.value,Adj.p.value,Description
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,R-HSA-1280218,816,10081887,0.1,0.7060,NA
2,R-HSA-5653656,710,8699864,0.1,0.7060,NA
3,R-HSA-983169,387,5007410,0.1,0.7060,NA
4,R-HSA-3700989,370,4586424,0.1,0.7060,NA
5,R-HSA-983168,313,4181108,0.1,0.7060,NA
6,R-HSA-195721,328,4150579,0.1,0.7060,NA


<!-- headings -->
<a id="padog-main"></a>
## 7. Enrichment analysis using PADOG

Pathway Analysis with Down-weighting of Overlapping Genes (PADOG) is a specialized method for enrichment analysis. Focused on deciphering the functional implications of differentially expressed genes in biological pathways, PADOG uniquely emphasizes the uniqueness of genes. Its down-weighting mechanism enhances precision by addressing challenges posed by overlapping genes, offering valuable insights into specific pathways influenced by gene expression changes.

To perform PADOG enrichment analysis, we will use the "PADOG" package. We first need to install and load the package.

In [160]:
suppressWarnings({
    suppressMessages({
        if (!require("BiocManager", quietly = TRUE)) {
            install.packages("BiocManager")
        }
        BiocManager::install("PADOG")
    })
})

suppressWarnings({suppressMessages({library(PADOG)})})

nperms <- 2000

We will use the `runPadog` function below to perform enrichment analysis using `PADOG` package. This function requires a gene expression matrix, a list of gene sets to analyze and the group information of the dataset. It will return the analysis results in a dataframe.

In [161]:
#' @description This function performs enrichment analysis using the PADOG method.
#'
#' @param exprsData A numeric matrix or a matrix-like object containing gene expression data. Rows represent genes, columns represent samples.
#' @param genesets A list of gene sets to be analzyed.
#' @param groups A factor indicating sample groups.
#' @param nperms An integer specifying the number of permutations for assessing significance (default is 100).
#' @param seed An integer specifying a seed for reproducibility of permutation results (default is 1).
#' @param ncores An integer specifying the number of cores to use for parallel processing (default is 100).
#'
#' @return A data frame containing pathway IDs and corresponding p-values.

runPadog <- function(exprsData, geneSet, groups, perm = 100, seed = 1, ncores = 1) {
    # Run PADOG analysis
    res <- PADOG::padog(
        esetm = as.matrix(exprsData),
        group = groups,
        gslist = geneSet,
        annotation = NULL,
        gs.names = names(geneSet),
        NI = nperms,
        plots = FALSE,
        targetgs = NULL,
        dseed = seed,
        parallel = TRUE,
        ncr = ncores
    )
    # Extract relevant columns and rename
    res <- res[, c('ID', 'Ppadog')]
    colnames(res) <- c("pathway", "p.value")
    
    return(res)
}


The PADOG method requires inputs that are quite similar to the GSA method. Thus, to perform analysis using PADOG, we will use the previously preprocessed `new_expression_data` dataframe and `groups` information for the GSA method above.

### Enrichment analysis using PADOG and GO terms

In [162]:
# Assign the geneset variable to GO_term_hallmark loaded in previous code
geneset <- GO_term_hallmark
# Perform PADOG enrichment analysis using the runPadog function
PADOG_result <- runPadog(exprsData = new_expression_data, geneSet = geneset, 
                         groups = groups, perm = nperms, seed = 1, ncore = ncores)

head(PADOG_result)

### Enrichment analysis using PADOG and KEGG pathways

We can use the same procedure to perform enrichment analysis with the GO terms. All the codes are similar but the gene sets list is loaded from `KEGG`. The code is shown below.

In [163]:
# Assign the geneset variable to KEGG_hallmark loaded in previous code
geneset <- KEGG_hallmark
# Perform PADOG enrichment analysis using the runPadog function
PADOG_result <- runPadog(exprsData = new_expression_data, geneSet = geneset, 
                         groups = groups, perm = nperms, seed = 1, ncore = ncores)

head(PADOG_result)

### Enrichment analysis using PADOG and REACTOME pathways

We will apply the same code that has been used for the GO terms and KEGG pathways.

In [164]:
# Assign the geneset variable to REACTOME_hallmark loaded in previous code
geneset <- REACTOME_hallmark
# Perform PADOG enrichment analysis using the runPadog function
PADOG_result <- runPadog(exprsData = new_expression_data, geneSet = geneset, 
                         groups = groups, perm = nperms, seed = 1, ncore = ncores)

head(PADOG_result)

The next submodule will go indepth about meta analysis.

In [166]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-apple-darwin20 (64-bit)
Running under: macOS Ventura 13.6.4

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.3-x86_64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.3-x86_64/Resources/lib/libRlapack.dylib;  LAPACK version 3.11.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/Chicago
tzcode source: internal

attached base packages:
[1] stats4    parallel  stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] PADOG_1.44.0             KEGGdzPathwaysGEO_1.40.0 GO.db_3.18.0            
 [4] doRNG_1.8.6              rngtools_1.5.2           hgu133plus2.db_3.13.0   
 [7] org.Hs.eg.db_3.18.0      safe_3.42.0              SparseM_1.81            
[10] AnnotationDbi_1.64.1     IRanges_2.36.0           S4Vectors_0.40.2        
[13] Biobase_2.62.0           BiocGenerics_0.48.1      doParallel